# VGG16

In [1]:
# create default vgg16
from tensorflow.keras.applications.vgg16 import VGG16

vgg16_model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')
vgg16_model.summary()

2023-05-08 11:27:28.327688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-08 11:27:29.109889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-08 11:27:29.110647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-08 11:27:29.128105: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Implement VGG16 by layers

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

### vgg16_default

In [3]:
def get_vgg16_default(input_shape=(224, 224, 3), classes=1000):
    inputs = Input(shape=input_shape, name="input_layer")
    
    # Block 1
    x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same", name="block1_conv1")(inputs)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same", name="block1_conv2")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool")(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same", name="block2_conv1")(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same", name="block2_conv2")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool")(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv1")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv2")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool")(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv1")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv2")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool")(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv1")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv2")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool")(x)

    # Classification block
    x = Flatten(name="flatten")(x)
    x = Dense(units=4096, activation="relu", name="fc1")(x)
    x = Dense(units=4096, activation="relu", name="fc2")(x)
    
    outputs = Dense(units=classes, activation="softmax", name="predictions")(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="vgg16_default")
    
    return model

In [4]:
vgg_default = get_vgg16_default()
vgg_default.summary()

Model: "vgg16_default"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)     

### vgg16 with global average pooling

In [5]:
def get_vgg16_avg(input_shape=(224, 224, 3), classes=10):
    inputs = Input(shape=input_shape, name="input_layer")
    
    # Block 1
    x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same", name="block1_conv1")(inputs)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same", name="block1_conv2")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool")(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same", name="block2_conv1")(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same", name="block2_conv2")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool")(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv1")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv2")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same", name="block3_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool")(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv1")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv2")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block4_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool")(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv1")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv2")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same", name="block5_conv3")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool")(x)

    # Classification block
    x = GlobalAveragePooling2D(name="global_avg_pool")(x)
    x = Dropout(rate=0.5, name="dropout_1")(x)
    x = Dense(units=128, activation="relu", name="fc1")(x)
    x = Dropout(rate=0.5, name="dropout_2")(x)
    
    outputs = Dense(units=classes, activation="softmax", name="predictions")(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="vgg16_avg")
    
    return model

In [6]:
vgg16_avg = get_vgg16_avg()
vgg16_avg.summary()

Model: "vgg16_avg"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0 